In [14]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision.utils import draw_segmentation_masks,save_image,make_grid
from torch.nn.functional import one_hot


img_idxes = [4208,2393]
batches =  []
imgs = torch.from_numpy(np.load('project_conic/CoNIC_Challenge/images.npy').astype(np.float64)/255).float().permute(0,3,1,2).contiguous()
labels = torch.from_numpy(np.load('project_conic/CoNIC_Challenge/labels.npy').astype(np.float64)).long().permute(0,3,1,2).contiguous()
color = torch.tensor([[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1]],dtype=torch.float32,device='cuda')
for img_idx in img_idxes:
    mask_map = labels[img_idx,1].cuda()
    instance_map = labels[img_idx,0].cuda()
    #(3,h,w)
    mask = color[mask_map].permute(2,0,1)
    #(num,h,w)
    instance_map = one_hot(instance_map,num_classes=instance_map.max()+1).permute(2,0,1)[1:].bool()

    background = torch.zeros(3,256,256,dtype=torch.uint8)
    instance =draw_segmentation_masks(background,instance_map,alpha=0.7)/ 255
    batches.append(imgs[img_idx])
    batches.append(instance)
    batches.append(mask.cpu())
batches =make_grid(batches,3)
save_image(batches ,'testpic2.png')

In [3]:
import numpy as np
import torch
from torchvision.utils import draw_segmentation_masks,save_image,make_grid
from torch.nn.functional import one_hot

img_idxes = [2393]
batches =  []
imgs = torch.from_numpy(np.load('project_conic/CoNIC_Challenge/images.npy').astype(np.float64)/255).float().permute(0,3,1,2).contiguous()
labels = torch.from_numpy(np.load('project_conic/CoNIC_Challenge/labels.npy').astype(np.float64)).long().permute(0,3,1,2).contiguous()
color = torch.tensor([[0,0,0],[1,0,0],[0,1,0],[0,0,1],[1,1,0],[1,0,1],[0,1,1]],dtype=torch.float32,device='cuda')

for idx in img_idxes:
    mask_map = labels[idx,1].cuda()
    instance_map = labels[idx,0].cuda()
    mask = color[mask_map].permute(2,0,1).cpu()
    instance_map = one_hot(instance_map,num_classes=instance_map.max()+1).permute(2,0,1)[1:].bool()
    instance = draw_segmentation_masks((imgs[idx]*255).to(dtype=torch.uint8),instance_map,0.7)/255
    pic=make_grid([instance,mask],1)
    save_image(pic,'testpic.png')